![alt text](<https://www.et-gw.eu/images/et-new-logo.png>)

#  XIII ET Symposium Hakathon session


#### Tutorial on ``gwfast``

This notebook is a simple tutorial to start playing around with the ``gwfast`` software, which is suitable for SNR and Fisher-matrix based parameter estimation forecasts on large catalogues of events.



## Installation for Google Colab

In [ ]:
#! pip install -q 'git+https://github.com/CosmoStatGW/gwfast' 

In [ ]:
#! pip install -q corner

**Note**: Using Google Colab, you need to restart the kernel runtime after installation.

## Now import some packages

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import h5py

import copy
from astropy.cosmology import Planck18

In [2]:
from gwfast import gwfastGlobals as glob

from gwfast.waveforms import IMRPhenomD_NRTidalv2, TaylorF2_RestrictedPN
from gwfast.signal import GWSignal
from gwfast.network import DetNet
from gwfast.fisherTools import CovMatr, compute_localization_region, check_covariance, fixParams
from gwfast import gwfastUtils as utils

## COMPLETE EXAMPLE: GW170817 @ ET!

###  We use the Sardinia site for definiteness

In [ ]:
alldetectors = copy.deepcopy(glob.detectors)
print('All available detectors are: '+str(list(alldetectors.keys())))

# select only ET in Sardinia
ETSdet = {det:alldetectors[det] for det in ['ETS']}
print('Using detector '+str(list(ETSdet.keys())))


In [4]:
# We use the ET-D psds
ETSdet['ETS']['psd_path'] = os.path.join(glob.detPath, 'ET-0000A-18.txt')

### Initialise the signals and then the network 
(in this case we have a single triangular detector, but we will see later why this syntax is useful)

In [ ]:
mywf = IMRPhenomD_NRTidalv2(fRef=50.)

mySignals = {}

for d in ETSdet.keys():

    mySignals[d] = GWSignal(mywf, 
                psd_path=ETSdet[d]['psd_path'],
                detector_shape = ETSdet[d]['shape'],
                det_lat= ETSdet[d]['lat'],
                det_long=ETSdet[d]['long'],
                det_xax=ETSdet[d]['xax'], 
                verbose=False,
                useEarthMotion = True,
                fmax=1024.,
                fmin=2.) 
        
myNet = DetNet(mySignals, verbose=False)      

### Now we build a dictionary containing the parameters of GW170817

In [ ]:
# Median values of the posterior samples for all the parameters, 
# except psi and the coalescence phase that are set to 0

z = np.array([0.00980])
tGPS = np.array([1187008882.4])

GW170817 = {'Mc':np.array([1.1859])*(1.+z), 
            'dL':Planck18.luminosity_distance(z).value/1000., 
            'theta':np.array([np.pi/2. + 0.4080839999999999]), 
            'phi':np.array([3.4461599999999994]),
            'iota':np.array([2.545065595974997]), 
            'psi':np.array([0.]),
            'tcoal':utils.GPSt_to_LMST(tGPS, lat=0., long=0.), # GMST is LMST computed at long = 0° 
            'eta':np.array([0.24786618323504223]), 
            'Phicoal':np.array([0.]), 
            'chi1z':np.array([0.005136138323169717]), 
            'chi2z':np.array([0.003235146993487445]), 
            'Lambda1':np.array([368.17802383555687]), 
            'Lambda2':np.array([586.5487031450857])
           }
GW170817['LambaTilde'], GW170817['deltaLambda'] = utils.Lamt_delLam_from_Lam12(GW170817['Lambda1'], GW170817['Lambda2'], GW170817['eta'])
print('Parameters for GW170817 are:')
GW170817

### Let's see how the signal looks like at one of the interferometers

In [ ]:
# first define the frequency grid
fmax = mywf.fcut(**GW170817)-10 # We do not get to the cut this way
fgrid = np.geomspace(1.,fmax,10000)

AmplatET_p, AmplatET_c = myNet.signals['ETS'].GWAmplitudes(GW170817, fgrid, rot=120.)

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))

ax.plot(fgrid, 2.*np.sqrt(fgrid)*np.sqrt(AmplatET_p**2 + AmplatET_c**2), linewidth=2., label='GW170817')
ax.plot(myNet.signals['ETS'].strainFreq, np.sqrt(myNet.signals['ETS'].noiseCurve), color='C2', linewidth=2., label='ET-D ASD')

ax.set_xlim(1.,fmax)
ax.set_ylim(1e-25, 1e-20)
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('f [Hz]', fontsize=15)
ax.set_ylabel(r'$2 \ |\tilde{h}| \ \sqrt{f}\ [Hz^{-1/2}]$', fontsize=15)
plt.grid(linestyle='dotted', linewidth='0.6', which='both')
ax.legend(loc='upper right', fontsize=15, ncol=1, fancybox=True)

plt.show()

In [ ]:
# Compute the time to coalescence
ft = 2.
t_to_coal = mywf.tau_star(ft,**GW170817)
print('The time to coalescence at %d Hz is %.0f hours!'%(ft, t_to_coal/(3600.)))

In [ ]:
# Plot the time to coalescence
fig, ax = plt.subplots(figsize=(12,6))
conv_to_hours = 3600.
ax.plot(fgrid[:,0], mywf.tau_star(fgrid[:,0],**GW170817) / conv_to_hours, linewidth=2.)

ax.set_xlim(1.,fgrid[:,0][-1])
ax.set_xscale('log')
ax.set_xlabel('f [Hz]', fontsize=15)
ax.set_ylabel(r'time to coalescence [hours]', fontsize=15)
plt.grid(linestyle='dotted', linewidth='0.6', which='both')

plt.show()

### Compute the expected matched-filter SNR

In [ ]:
SNR = myNet.SNR(GW170817)
print('SNR for GW170817 at ET is %.2f'%SNR[0])

### Compute the total Fisher matrix

In [ ]:
totF = myNet.FisherMatr(GW170817)
print('The computed Fisher matrix has shape %s'%str(totF.shape))

In [ ]:
# Check e.g. that the (dL,dL) element corresponds to (SNR/dL)^2
ParNums = mywf.ParNums
dL_Num = ParNums['dL']
print('The relative difference is %.2e !'%((1 - totF[ParNums['dL'],ParNums['dL'],:]/(SNR/GW170817['dL'])**2)[0]))


### Compute the covariance and perform some checks

In [11]:
totCov, inversion_err = CovMatr(totF)


In [ ]:
_ = check_covariance(totF, totCov)


### Now try to eliminate the row corresponding to $\delta\tilde{\Lambda}$, and see that the inversion error lowers

In [ ]:
ParNums = mywf.ParNums

newFish, newPars = fixParams(totF, ParNums, ['deltaLambda'])

print('Now the Fisher matrix has shape %s'%str(newFish.shape))

newCov, new_inversion_err = CovMatr(newFish)

_ = check_covariance(newFish, newCov)


### Finally compute the localisation region

In [ ]:
skyArea = compute_localization_region(newCov, newPars, GW170817['theta'])
print('The estimated sky location is %.2f deg^2'%skyArea)


### Let's make a corner plot to visualize the result

In [ ]:
from corner import corner

In [ ]:
CORNER_KWARGS = dict(
    bins = 50, # number of bins for histograms
    smooth = 0.99, # smooths out contours. 
    plot_datapoints = True, # choose if you want datapoints
    label_kwargs = dict(fontsize = 12), # font size for labels
    show_titles = True, #choose if you want titles on top of densities.
    title_kwargs = dict(fontsize = 12), # font size for title
    plot_density = False,
    title_quantiles = [0.16, 0.5, 0.84],  # add quantiles to plot densities for 1d hist
    levels = (1 - np.exp(-0.5), 1 - np.exp(-2), 1 - np.exp(-9 / 2.)), # 1, 2 and 3 sigma contours for 2d plots
    fill_contours = True, #decide if you want to fill the contours
    max_n_ticks = 2, # set a limit to ticks in the x-y axes.
    title_fmt=".3f"
    )
corner_lbs = [r'${\cal M}_c$ $[M_{\odot}]$', '$\eta$', '$d_L$ [Gpc]', '$\theta$ $[rad]$', '$\phi$ $[rad]$', 
                '$\iota$ [rad]', '$\psi$ [rad]', '$t_c$ [s]', '$\Phi_c$ [rad]', '$\chi_{1,z}$', '$\chi_{1,z}$', '$\\tilde{\Lambda}$']
mean_values = [GW170817['Mc'][0], GW170817['eta'][0], \
               GW170817['dL'][0], GW170817['theta'][0], GW170817['phi'][0], \
               GW170817['iota'][0], GW170817['psi'][0], GW170817['tcoal'][0], GW170817['Phicoal'][0], \
               GW170817['chi1z'][0], GW170817['chi2z'][0], GW170817['LambaTilde'][0]]

# Sample from a multi-variate gaussian with the given covariance matrix and injected mean values
samples = np.random.multivariate_normal(mean_values, newCov[:,:,0], int(1e5))
fig = corner(samples, labels = corner_lbs, truths = mean_values, truth_color = 'red',
                    **CORNER_KWARGS)
plt.show()

## COMPLETE EXAMPLE: a BNS population @ ET!


### Load the data (if you are curious about the generation, just ask!)

In [ ]:
import pandas as pd

data_url = 'https://raw.githubusercontent.com/FrancescoIacovelli/XIII_ET_Symposium_Hakathon/main/data/BNS_100ev_pop.csv'

df = pd.read_csv(data_url, delimiter=',')
df

In [ ]:
# Translate into a dictionary for GWFAST
evsdict = df.to_dict('list')
for key in evsdict.keys():
  evsdict[key] = np.array(evsdict[key])

### Switch to TaylorF2 to save some time

In [ ]:
mywf = TaylorF2_RestrictedPN(is_tidal=True, use_3p5PN_SpinHO=True, use_QuadMonTid=True)

mySignals = {}

for d in ETSdet.keys():

    mySignals[d] = GWSignal(mywf, 
                psd_path=ETSdet[d]['psd_path'],
                detector_shape = ETSdet[d]['shape'],
                det_lat= ETSdet[d]['lat'],
                det_long=ETSdet[d]['long'],
                det_xax=ETSdet[d]['xax'], 
                verbose=False,
                useEarthMotion = True,
                fmax=1024.,
                fmin=2.) 
        
myNet = DetNet(mySignals, verbose=False) 

### Let's compute the SNRs and plot them

In [ ]:
SNRs_pop = myNet.SNR(evsdict)

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))

_ = ax.hist(SNRs_pop, linewidth=2., label='SNRs at ET', bins=51, histtype='step')

ax.set_xlim(0.,150.)
#ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('SNR', fontsize=15)
ax.set_ylabel(r'Number of events', fontsize=15)
plt.grid(linestyle='dotted', linewidth='0.6', which='both')
plt.axvline(12, linestyle='--', color='k')
ax.legend(loc='upper right', fontsize=15, ncol=1, fancybox=True)

plt.show()

print('There are %d events with SNR>12'%len(SNRs_pop[SNRs_pop>12]))

### Now we select the events with SNR>12 and compute the Fishers

In [ ]:
mask_det = SNRs_pop>12.

evs_det = {}

for key in evsdict.keys():
  evs_det[key] = evsdict[key][mask_det]

In [ ]:
Fishers_pop = myNet.FisherMatr(evs_det)
print('The computed Fisher matrices have shape %s'%str(Fishers_pop.shape))

In [ ]:
# We again fix delta Lambda
ParNums = mywf.ParNums

newFish_pop, newPars = fixParams(Fishers_pop, ParNums, ['deltaLambda'])

print('Now the Fisher matrices have shape %s'%str(newFish_pop.shape))

Cov_pop, inversion_err_pop = CovMatr(newFish_pop)

print('The maximum inversion error is %d%%'%int(max(inversion_err_pop)*100))

In [ ]:
# Get errors and sky localisations

errors_pop = np.array([np.sqrt(Cov_pop[i, i]) for i in range(Cov_pop.shape[0])])
skyArea_pop = compute_localization_region(Cov_pop, newPars, evs_det['theta'])

### Finally plot!

In [ ]:
whole_sky = 4.*(180.**2)/np.pi
cm = matplotlib.colormaps.get_cmap('inferno_r')

fig, ax = plt.subplots(figsize=(12,6))

sc = ax.scatter(errors_pop[newPars['dL'],:]/evs_det['dL'], skyArea_pop, c = SNRs_pop[SNRs_pop>12], cmap = cm)

ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlim(1e-1, 1e1)
ax.set_ylim(1e1, 1e5)
ax.set_xlabel('$\Delta d_L/d_L$', fontsize=15)
ax.set_ylabel('$\Delta \Omega_{90}$ $[deg^2]$', fontsize=15)
plt.colorbar(sc, label = 'SNR')
sc.figure.axes[1].yaxis.label.set_size(15)

plt.grid(linestyle='dotted', linewidth='0.6', which='both')
plt.axhline(whole_sky, linestyle='--', color='k')

plt.show()